In [1]:
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, TimeDistributed, LSTM, Conv1D
from keras.layers import Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
C:\Users\epour\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
num_set = 15               # number of training sets
num_train = 100            # number of samples in each training set
Tx = 5511                  # number of spectrogram time steps in the input
n_freq = 101               # number of frequencies in the spectrogram
Ty = 1375                  # number of time steps in the output of our model

X = np.zeros((num_set*num_train, Tx, n_freq))
Y = np.zeros((num_set*num_train, Ty, 1))

for i in range(num_set):
    X[i*num_train:(i+1)*num_train,:,:] = np.load("./train/X_train" + str(i+1) + ".npy")
    Y[i*num_train:(i+1)*num_train,:,:] = np.load("./train/Y_train" + str(i+1) + ".npy")

X_test = np.load("./train/X_test.npy")
Y_test = np.load("./train/Y_test.npy")

In [3]:
def Model():

    model = Sequential()
    
    # layer 1: convolutional layer
    model.add(Conv1D(196, kernel_size=15, strides=4))           # CONV1D
    model.add(BatchNormalization(axis = -1))                    # Batch normalization
    model.add(Activation('relu'))                               # ReLu activation
    model.add(Dropout(0.2))                                     # dropout 

    # layer 2: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))        # LSTM
    model.add(Dropout(0.2))                                     # dropout
    model.add(BatchNormalization(axis = -1))                    # Batch normalization
    
    # layer 3: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))        # LSTM
    model.add(Dropout(0.2))                                     # dropout 
    model.add(BatchNormalization(axis = -1))                    # Batch normalization
    
    # layer 4: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))        # LSTM
    model.add(Dropout(0.2))                                     # dropout 
    model.add(BatchNormalization(axis = -1))                    # Batch normalization
    
    # layer 5: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))        # LSTM
    model.add(Dropout(0.2))                                     # dropout 
    model.add(BatchNormalization(axis = -1))                    # Batch normalization
    model.add(Dropout(0.2))                                     # dropout 
    
    # layer 6: dense layer with sigmoid activation
    model.add(TimeDistributed(Dense(1, activation = "sigmoid"))) # dense & sigmoid
    
    return model 

In [4]:
model = Model()                                                                 # build a model
opt = Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, decay=0.01)                     # set the optimizer
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])  # compile the model
model.fit(X, Y, batch_size = 64, epochs=30)                                     # fit the model






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
1500/1500 [==============================] - 228s 152ms/step - loss: 0.7509 - acc: 0.5513
Epoch 2/30
1500/1500 [==============================] - 197s 131ms/step - loss: 0.5844 - acc: 0.7373
Epoch 3/30
1500/1500 [==============================] - 192s 128ms/step - loss: 0.4497 - acc: 0.8715
Epoch 4/30
1500/1500 [==============================] - 190s 127ms/step - loss: 0.3266 - acc: 0.9493
Epoch 5/30
1500/1500 [==============================] - 190s 127ms/step - loss: 0.2393 - acc: 0.9796
Epoch 6/30
1500/1500 [==============================] - 191s 127ms/step - loss: 0.1978 - acc: 0.9860
Epoch 7/30
1500/1500 [==============================] - 191s 127ms/step - loss: 0.1584 - acc: 0.9908
Epoch 8/30
1500/1500 [==============================] - 197s 131ms/step - loss

In [5]:
_, acc_train = model.evaluate(X, Y)             # evaluate accuracy on training set
print("Train set accuracy = ", acc_train)

1500/1500 [==============================] - 215s 143ms/step
Train set accuracy =  0.9976882403691609


In [6]:
_, acc_test = model.evaluate(X_test, Y_test)    # evaluate accuracy on test set
print("Test set accuracy = ", acc_test)

100/100 [==============================] - 18s 177ms/step
Test set accuracy =  0.997803635597229


In [7]:
model.save("model.h5")                         # save the model